In [12]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import MultiTaskLasso
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SequentialFeatureSelector

In [13]:
data = pd.read_csv("parser_output/extracted_features_split_upsample_3_7.csv")
data = data.dropna()

In [14]:
data

,Unnamed: 0,Unnamed: 0.1,LEFT_EYE_OUTER_POS_x__abs_energy,LEFT_EYE_OUTER_POS_x__kurtosis,LEFT_EYE_OUTER_POS_x__mean,LEFT_EYE_OUTER_POS_x__standard_deviation,LEFT_EYE_OUTER_POS_x__maximum,LEFT_EYE_OUTER_POS_x__autocorrelation__lag_1,LEFT_EYE_OUTER_POS_x__approximate_entropy__m_20__r_0.05,LEFT_EYE_OUTER_POS_x__c3__lag_1,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0,1.000000,3.857886e+06,-0.500597,229.468110,30.432529,289.405029,0.989740,0.019048,1.261402e+07,...,1.000000,-11.683000,1.000000,0.471000,0.034600,0.069800,0.112000,0.394000,120.461000,4.000000
1,1,1.000000,3.855130e+06,-0.483415,231.389670,27.503200,295.279053,0.958905,0.019418,1.296885e+07,...,1.000000,-11.683000,1.000000,0.471000,0.034600,0.069800,0.112000,0.394000,120.461000,4.000000
2,2,1.000000,2.988295e+06,-1.564327,204.247290,19.279543,231.353989,0.984248,0.019418,8.737854e+06,...,1.000000,-11.683000,1.000000,0.471000,0.034600,0.069800,0.112000,0.394000,120.461000,4.000000
3,3,1.000000,5.229781e+06,5.505027,261.979358,13.406352,288.362549,0.740990,0.017700,1.824470e+07,...,1.000000,-4.961000,1.000000,0.040600,0.013000,0.000000,0.094400,0.860000,134.066000,4.000000
4,4,1.000000,3.804110e+06,-1.651266,221.742280,29.739429,267.973663,0.980201,0.017700,1.132808e+07,...,1.000000,-4.961000,1.000000,0.040600,0.013000,0.000000,0.094400,0.860000,134.066000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,2731,1.004642,7.675702e+06,-0.428105,308.844848,23.695611,354.531617,0.934245,0.017084,2.980213e+07,...,1.008252,-8.051201,0.005007,0.068594,0.435179,0.004971,0.121395,0.323141,110.026127,4.008011
2732,2732,1.002437,6.251677e+06,-0.295028,278.277536,26.620029,340.216678,0.970170,0.019958,2.204985e+07,...,1.002027,-8.045381,0.004618,0.063657,0.442621,0.003104,0.117410,0.325989,110.020294,4.003570
2733,2733,1.004247,6.060124e+06,-0.447082,262.692938,38.227663,341.837440,0.977176,0.021461,1.917193e+07,...,2.006992,-2.717840,0.006860,0.147851,0.099990,0.004684,0.294257,0.717822,120.028234,4.001796
2734,2734,1.005076,7.223848e+06,-1.371630,289.099607,37.607511,361.694106,1.004394,0.019952,2.539007e+07,...,2.000985,-2.712416,0.004196,0.147485,0.103294,0.007745,0.294076,0.721028,120.036452,4.007549


In [15]:
y = data[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
x = data[data.columns[1:len(data.columns)-12]]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.7)

In [16]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [17]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

# ypred_train = rf_random.predict(X_train)
# ypred_test = rf_random.predict(X_test)
# print("RANDOM FOREST REGRESSION")
# print("Train Score: ")
# print(r2_score(y_train, ypred_train))
# print("Test Score: ")
# print(r2_score(y_test, ypred_test))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [18]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [19]:
# Compare to original

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy



base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(train_features, train_labels)
base_accuracy = evaluate(base_model, test_features, test_labels)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test_features, test_labels)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

NameError: name 'train_features' is not defined